In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import sklearn
import scipy as sp

In [227]:
# Data processing into features and labels
data = pd.read_csv("HW2_data.csv")
names = data.columns[0:-1].tolist()
genes = data.shape[1]-1
patients = data.shape[0]

In [226]:
# Entropy calculation, defaults to 0 if log cannot be found
def entropy(a,b):
    if (a+b) == 0:
        return(0)
    p1,p2 = a/(a+b),b/(a+b)
    term1,term2 = 0,0
    if p1 != 0:
        term1 = p1 * np.log2(p1)
    if p2 != 0:
        term2 = p2 * np.log2(p2)
    return(-term1 - term2)

# Iterates through each gene, finding threshold with maximum information gain
# Uses values to discretize original data
column_list = []
gain = []
for i in range(genes):
    gene_data = data.iloc[:,[i,-1]]
    sort = gene_data.sort_values(gene_data.columns[0])
    ig_list = []
    for j in range(patients + 1):
        group1 = sort.iloc[:j,:]
        a1,b1 = len(group1[group1.prognosis == 'GOOD']),len(group1[group1.prognosis == 'POOR'])
        group2 = sort.iloc[j:,:]
        a2,b2 = len(group2[group2.prognosis == 'GOOD']),len(group2[group2.prognosis == 'POOR'])
        ig_list.append(entropy(62,25) - ((j/patients)*entropy(a1,b1)) - (((patients-j)/patients)*entropy(a2,b2)))
    th_index = ig_list.index(max(ig_list))
    th = sort.iloc[th_index,0]
    column = data.iloc[:,i] >= th
    column_list.append(column.astype(int))
    gain.append(max(ig_list))
column_list.append(data.iloc[:,-1])
discretized = pd.concat(column_list,axis=1)

In [214]:
# Divide data into good and poor subsets
ood_list = []
poor_list = []
good = discretized[discretized.prognosis == 'GOOD']
poor = discretized[discretized.prognosis == 'POOR']

# Correction for log if attempting log(0)
def new_log(x):
    if x == 0:
        return(np.log(0.001))
    else:
        return(np.log(x))

# Find the probability of expression being 1 for each gene in each subsetna
for i in names:
    good_list.append(np.mean(good[i]))
    poor_list.append(np.mean(poor[i]))

# Naive Bayes implementaiton for classification. 
def predict_class(gene_list):
    classes = []
    class_prob = []
    for i in range(patients):
        p_good = np.log(62/87)
        p_poor = np.log(25/87)
        for j in gene_list:
            if discretized.iloc[i,j] == 1:
                p_good += new_log(good_list[j])
                p_poor += new_log(poor_list[j])
            else:
                p_good += new_log(1-good_list[j])
                p_poor += new_log(1-poor_list[j])
        class_prob.append(p_good-p_poor)
        classes.append((p_good-p_poor) >= 0)
    print(sum(classes == (discretized.prognosis == 'GOOD'))/87)
    return(class_prob)

In [228]:
# Sorted genes by information gain, and calclate accuracy of prediction
gene_ig = pd.DataFrame({"gene":names,"ig":gain})
gene_ig_sorted = gene_ig.sort_values(by=["ig"],ascending=False)
print(gene_ig_sorted)

genes_3 = predict_class(list(gene_ig_sorted.iloc[0:3,:].index))
genes_5 = predict_class(list(gene_ig_sorted.iloc[0:5,:].index))
genes_10 = predict_class(list(gene_ig_sorted.iloc[0:10,:].index))
genes_all = predict_class(list(range(30)))

          gene        ig
6     33662_at  0.199922
23    38839_at  0.193551
13    33809_at  0.168857
25  38840_s_at  0.164204
10  33672_f_at  0.159296
7     33713_at  0.155248
8     41172_at  0.155059
3   33582_s_at  0.153269
18    34683_at  0.142871
20  31989_s_at  0.132685
28  31377_r_at  0.132413
29    37439_at  0.131325
19    31578_at  0.131325
24    35196_at  0.130524
17    41044_at  0.124735
5   36559_g_at  0.123653
4     40131_at  0.122553
22    38221_at  0.122079
1     35291_at  0.122079
21    32607_at  0.120237
0     37086_at  0.116993
15    32672_at  0.115832
14    39298_at  0.110590
9     39028_at  0.102109
2     41757_at  0.101468
26    41872_at  0.095960
16    32434_at  0.093014
11    35895_at  0.091058
27    33741_at  0.091058
12  38673_s_at  0.080427
0.8275862068965517
0.8045977011494253
0.8850574712643678
0.9655172413793104


In [220]:
# Calculate AUC values
from sklearn.metrics import roc_curve, roc_auc_score
print(roc_auc_score(discretized.prognosis == "GOOD",genes_3))
print(roc_auc_score(discretized.prognosis == "GOOD",genes_5))
print(roc_auc_score(discretized.prognosis == "GOOD",genes_10))
print(roc_auc_score(discretized.prognosis == "GOOD",genes_all))

0.8641935483870968
0.9009677419354839
0.960967741935484
0.9974193548387097
